<a href="https://colab.research.google.com/github/SergeyKamenshchikov/CNTR-recongition/blob/main/accelerator_%D0%9D%D0%B0%D1%83%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Install libraries

In [1]:
!pip3 install openai==1.55.3 -q
!apt-get install poppler-utils

!apt-get update && apt-get install -y pandoc
!pip3 install python-pptx pandas tabulate
!pip3 install pypandoc

!pip3 install pdfplumber
!pip3 install pdf2image
!pip3 install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 5.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 1s (158 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126441 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get

##### Import libraries:

In [2]:
from openai import OpenAI

import os, time, json

import numpy as np
import pandas as pd

from tqdm import tqdm

from pptx import Presentation

from google.colab import drive
from google.colab import output
from google.colab import files

from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display

import nltk, re
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

from tqdm import tqdm

from pdf2image import convert_from_path
import easyocr, pdfplumber

import warnings, logging

import threading, queue
from collections import namedtuple

import base64
import tempfile
import pypandoc

from PIL import Image

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


##### Ignore warnings

In [3]:
warnings.filterwarnings("ignore")
logging.getLogger("pdfminer").setLevel(logging.ERROR)

##### Keys

##### UDFs

In [5]:
# docx converters
def docx_to_plain_with_simple_tables(path: str) -> str:
    return pypandoc.convert_file(path, to="plain+simple_tables", format="docx")

def pptx_to_markdown(pptx_file):
    prs = Presentation(pptx_file)
    md_content = []

    for i, slide in enumerate(prs.slides, 1):
        md_content.append(f"## Slide {i}\n")

        for shape in slide.shapes:
            if shape.has_text_frame:
                for paragraph in shape.text_frame.paragraphs:
                    if paragraph.text.strip():
                        md_content.append(paragraph.text + "\n")

            elif shape.has_table:
                table = shape.table
                table_data = []
                for row in table.rows:
                    row_data = [cell.text.strip() for cell in row.cells]
                    table_data.append(row_data)

                if table_data:
                    df = pd.DataFrame(table_data[1:], columns=table_data[0])
                    md_content.append(df.to_markdown(index=False) + "\n\n")

        md_content.append("---\n\n")

    return ''.join(md_content)
# docx converters

# field extraction
def extract_field(text, prompt, counter=0, model="o3") -> dict:

    client = OpenAI()

    system_prompt = ("Ты эксперт по технологическим стартапам и смотришь презентацию стартапа для венчурного фонда")

    schema = {"name": "startup_analysis", "schema": {"type": "object", "properties": {"university": {"type": "string"}, "legal_name": {"type": "string"},\
                "region": {"type": "string"}, "project_name": {"type": "string"}, "product_description": {"type": "string"},
                "ugt": {"type": "string"}, "ip": {"type": "string"}, "team": {"type": "string"}, "bm": {"type": "string"},
                "market": {"type": "string"}, "roadmap": {"type": "string"}, "competitors": {"type": "string"},
                "investments": {"type": "string"}, "risks": {"type": "string"}},

            "required": ["university", "legal_name", "region", "project_name", "product_description", "ugt", "ip", "team", "bm", "market",
                "roadmap", "competitors", "investments", "risks"], "additionalProperties": False}, "strict": True}

    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f'task: {prompt} : "text: {text}"'}]
    response = client.chat.completions.create(model=model, messages=messages, response_format={"type": "json_schema", "json_schema": schema})

    return json.loads(response.choices[0].message.content)

def return_fields(doc_txt):

    result = extract_field(doc_txt, '''извлеки отсюда информацию в JSON формате:

        1. university: ВУЗ (высшее учебное заведение), который предлагает проект - если нет, выведи только одно слово 'НЕТ'
        2. legal_name: юридическое наименование компании, которая предлагает продукт в формате ООО или АО или ПАО и затем фраза в кавычках - если нет, выведи только одно слово 'НЕТ'
        3. region: регион, в котором работает компания или ВУЗ - если нет, выведи только одно слово 'НЕТ'
        4. project_name: название проекта - если нет, выведи только одно слово 'НЕТ'
        5. product_description: описание технологического продукта компании, с умпоминанием технологии и решаемой продуктом проблемы пользователя (максимум три предложения) - если нет, выведи только одно слово 'НЕТ'
        6. ugt: УГТ проекта (уровень технологической готовности) - может быть 'УГТ 1', 'УГТ 2', 'УГТ 3', 'УГТ 4', 'УГТ 5', 'УГТ 6', 'УГТ 7', 'УГТ 8', 'УГТ 9' - если нет, выведи только одно слово 'НЕТ'
        7. ip: интеллектуальная собственность и что удалось сделать на сегодня команде по проекту (максимум пять предложений) - если нет, выведи только одно слово 'НЕТ'
        8. team: команда проекта - ФИО ключевых сотрудников, описание их опыта и экспертизы, контакты ключевых сотрудников (максимум пять предложений) - если нет, выведи только одно слово 'НЕТ'
        9. bm: бизнес модель (максимум три предложения) - если нет, выведи только одно слово 'НЕТ'
        10. market: целевой рынок продукта с объемом и описанием (максимум три предложения) - если нет, выведи только одно слово 'НЕТ'
        11. roadmap: дорожная карта проекта с датами (максимум пять предложений) - если нет, выведи только одно слово 'НЕТ'
        12. competitors: чем лучше и хуже конкурентов и аналогов в России и в Мире (максимум пять предложений) - если нет, выведи только одно слово 'НЕТ'
        13. investments: требуемая сумма инвестиций для развития проекта (не от государства, не грант и не субсидия) - если нет, выведи только одно слово 'НЕТ'
        14. risks: указанные риски проекта - включая технологические, рыночные, правовые, операционные и финансовые (максимум три предложения) - если нет, выведи только одно слово 'НЕТ''')

    return result
#/field extraction

# tech classification
def nptl_class(text, counter=0, model="o3", system_prompt = None):

    client = OpenAI()

    system_prompt = "Ты эксперт по технологическим стартапам и смотришь презентацию стартапа для венчурного фонда"

    prompt = '''К какому одному ключевому направлению из списка 'Новые материалы и химия', 'Новые технологии сбережение здоровья',
    'Средства производства и автоматизации', 'Новые атомные и энергетические технологии', 'Экономика данных и цифровая трансформация',
    'Космос', 'Технологическое обеспечение биоэкономики', 'Беспилотные авиационные системы', 'Технологическое обеспечение продовольственной безопасности',
    'Промышленное обеспечение транспортной мобильности' относится этот текст, если ни к какому выведи 'Другое/не определено':'''

    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f'task: {prompt} : "text: {text}" '}]
    response = client.chat.completions.create(model=model, messages=messages).choices[0].message.content

    return response
#/tech classification

# extract technology
def extract_technology(text):

  system_prompt = "Ты эксперт по технологическим стартапам и смотришь презентацию стартапа для венчурного фонда"

  messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": 'Извлеки из этого описания ключевые технологические методы для работы продукта\
   - одна фраза, исключающая упоминание продукта и решаемых им задач (вез вводных слов)' + ": '" + text + "'"}]

  completion = OpenAI().chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0)
  content = completion.choices[0].message.content

  return str(content).replace('"','')
#/extract technology

# parallel computations
def parallel_extraction(texts, max_workers: int = 10):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(lambda x: return_fields(x), texts), total=len(texts)))
    return results

def parallel_text_func(texts, func, max_workers: int = 10):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(lambda x: func(x), texts), total=len(texts)))
    return results
#/parallel computations

##### Load decks

In [6]:
drive.mount('/content/drive')
!cp -r '/content/drive/My Drive/Pitchdecks' '/content/'

Mounted at /content/drive


##### Extract fields

In [7]:
%%time

df_fields, file_texts, adresses = [], [], []
folder_path = '/content/Pitchdecks'

### Extract .docx files
docx_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.docx')])

for docx_file in tqdm(docx_files):

    pptx_path = os.path.join(folder_path, docx_file)
    file_texts.append(docx_to_plain_with_simple_tables(pptx_path))
    adresses.append(pptx_path.replace('/content/Питчи/', ''))
###/Extract .docx files

### Extract .pptx files
pptx_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pptx')])

for pptx_file in tqdm(pptx_files):

    pptx_path = os.path.join(folder_path, pptx_file)
    file_texts.append(pptx_to_markdown(pptx_path))
    adresses.append(pptx_path.replace('/content/Питчи/', ''))
###/Extract .pptx files

### Extract .pdf files
pdf_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.pdf')])

for pdf_file in tqdm(pdf_files):

    pdf_path = os.path.join(folder_path, pdf_file)
    adresses.append(pdf_path.replace('/content/Питчи/', ''))

    # Extract text with PDF reader
    with pdfplumber.open(pdf_path) as pdf:
      text = ''
      for page in pdf.pages:
          text += page.extract_text() + '\n'

    doc_txt = text.replace('02.12.2024, 22:35 Документ от auto-coder.net', '')

    if not any(char.isalpha() for char in doc_txt):

        pdf_images = convert_from_path(pdf_path, thread_count=10)

        ###/Image recognition ###
        for idx, image in enumerate(pdf_images):
          image.save(f'pdf_page_{idx + 1}.png', 'PNG')

        image_dir = '/content/'
        reader = easyocr.Reader(['ru', 'en'], gpu = True)

        text_results = []

        for filename in sorted(os.listdir(image_dir)):

            if filename.lower().endswith('.png'):

              try:
                img_path = os.path.join(image_dir, filename)
                result = reader.readtext(img_path, detail=0, paragraph=True)
                text_results.extend(result)
              except:
                print("Can't recognize the image")

        img_text = '\n'.join(text_results)
        !rm /content/*.png
        ###/Image recognition ###

        file_texts.append('|Краткое содержание текста: ' + img_text)
    else:
        file_texts.append('|Краткое содержание текста: ' + doc_txt)
###/Extract .pdf files

df_fields = list(parallel_extraction(file_texts))
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

0it [00:00, ?it/s]
0it [00:00, ?it/s]
 93%|█████████▎| 42/45 [00:48<00:03,  1.13s/it]WARNING:easyocr.easyocr:Downloading detection model, please wait. This may take several minutes depending upon your network connection.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

100%|██████████| 45/45 [02:38<00:00,  3.51s/it]


CPU times: user 1min 41s, sys: 1.91 s, total: 1min 43s
Wall time: 7min 52s


##### Create dataframe:

In [8]:
df = pd.DataFrame(df_fields)

df.columns = ['Университет', 'ЮЛ', 'Регион присутствия', 'Название проекта', 'Описание проекта (в т.ч. продукт-решаемая проблема)', 'УГТ', 'Текущие достижения и IP',\
                                        'Команда проекта', 'Бизнес - модель', 'Целевой рынок', 'Дорожная карта / календарный план', 'Результаты конкурентного анализа', 'Требуемый объем инвестиций', 'Риски']

df['Название файла'] = adresses
df['Название файла'] = df['Название файла'].str.split('/').str[-1]

print(df.head().to_markdown(index=False, tablefmt="grid"))

+-------------------------------------------------+------+----------------------+------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Add tech focus and NPTL:

In [9]:
df['Технологический фокус'] = parallel_text_func(list(df['Описание проекта (в т.ч. продукт-решаемая проблема)'].astype(str)), extract_technology, 10)
df['НПТЛ'] = parallel_text_func(list(df['Описание проекта (в т.ч. продукт-решаемая проблема)'].astype(str)), nptl_class, 10)

df[['Технологический фокус', 'НПТЛ']] = df[['Технологический фокус', 'НПТЛ']].replace(r'\.', '', regex=True)
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

100%|██████████| 45/45 [00:16<00:00,  2.75it/s]


##### Save dataframe:

In [10]:
df = df[['Университет', 'ЮЛ', 'Регион присутствия', 'Название проекта', 'Описание проекта (в т.ч. продукт-решаемая проблема)', 'НПТЛ', 'УГТ',\
         'Технологический фокус', 'Текущие достижения и IP', 'Команда проекта', 'Бизнес - модель', 'Целевой рынок', 'Дорожная карта / календарный план',\
         'Результаты конкурентного анализа', 'Требуемый объем инвестиций', 'Риски']]

print(df.head(10).to_markdown(index=False, tablefmt="grid"))

+-------------------------------------------------+------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+------

##### Save to xlsx:

In [11]:
df.to_excel('science_pitches.xlsx', index=False)
files.download('science_pitches.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>